<a href="https://colab.research.google.com/github/Ruran8wa/sentio/blob/main/notebook/01_data_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!git clone https://github.com/Ruran8wa/sentio.git
%cd sentio/

Cloning into 'sentio'...
remote: Enumerating objects: 34060, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 34060 (delta 0), reused 0 (delta 0), pack-reused 34056 (from 2)
Receiving objects: 100% (34060/34060), 52.45 MiB | 27.14 MiB/s, done.
Resolving deltas: 100% (4/4), done.
Updating files: 100% (35889/35889), done.
/content/sentio/sentio/sentio


In [31]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping

In [32]:
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'

In [33]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

# Map class names to indices based on train_generator.class_indices
class_indices = train_generator.class_indices
print(class_indices)  # e.g. {'angry':0, 'disgust':1, ...}

# List counts in the order of indices
counts_ordered = [class_counts[cls] for cls in sorted(class_indices, key=class_indices.get)]

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.arange(len(classes)),
    y=np.concatenate([[i]*count for i, count in enumerate(counts_ordered)])
)

class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)

{'angry': 0, 'disgust': 1, 'fear': 2, 'happy': 3, 'neutral': 4, 'sad': 5, 'surprise': 6}
{0: np.float64(1.0266046844269623), 1: np.float64(9.406618610747051), 2: np.float64(1.0010460615781582), 3: np.float64(0.5684387684387684), 4: np.float64(0.8260394187886635), 5: np.float64(0.8491274770777877), 6: np.float64(1.293372978330405)}


In [34]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    shear_range=0.3,
    zoom_range=0.3,
    width_shift_range=0.4,
    height_shift_range=0.4,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2,
    dtype='float32'
)

val_test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical',
    subset='validation'
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    batch_size=32,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [35]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(48, 48, 1)),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(64, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Conv2D(128, (3, 3), activation='relu'),
    BatchNormalization(),
    MaxPooling2D((2, 2)),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(7, activation='softmax')  # 7 emotion classes
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [36]:
early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

In [37]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,
    callbacks=[early_stop],
    class_weight=class_weights_dict
)

Epoch 1/50


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


718/718 ━━━━━━━━━━━━━━━━━━━━ 119s 161ms/step - accuracy: 0.1061 - loss: 2.2214 - val_accuracy: 0.0503 - val_loss: 1.9506
Epoch 2/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 117s 162ms/step - accuracy: 0.0408 - loss: 1.9437 - val_accuracy: 0.0260 - val_loss: 1.9519
Epoch 3/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 117s 162ms/step - accuracy: 0.1045 - loss: 1.9473 - val_accuracy: 0.0186 - val_loss: 1.9508
Epoch 4/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 116s 162ms/step - accuracy: 0.0165 - loss: 1.9572 - val_accuracy: 0.0152 - val_loss: 1.9465
Epoch 5/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 117s 163ms/step - accuracy: 0.1208 - loss: 1.9347 - val_accuracy: 0.0209 - val_loss: 1.9468
Epoch 6/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 116s 162ms/step - accuracy: 0.1030 - loss: 1.9357 - val_accuracy: 0.2510 - val_loss: 1.9456
Epoch 7/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 116s 162ms/step - accuracy: 0.1433 - loss: 1.9470 - val_accuracy: 0.0155 - val_loss: 1.9464
Epoch 8/50
718/718 ━━━━━━━━━━━━━━━━━━━━ 116s 162ms/step - accuracy: 0.2044 - loss: 1.92

In [40]:
test_loss, test_acc = model.evaluate(test_generator)
print(f"\n✅ Test Accuracy: {test_acc:.2f}")

225/225 ━━━━━━━━━━━━━━━━━━━━ 9s 40ms/step - accuracy: 0.2450 - loss: 1.9443

✅ Test Accuracy: 0.25
